In [1]:
import pandas as pd
import numpy as np

pd.__version__

'0.23.1'

## Create

In [ ]:
# Create empty dataframe
df = pd.DataFrame()

# Create from CSV
df = pd.read_csv('./data/file.csv')

# Create from random values
df = pd.DataFrame(np.random.randn(5, 3),
                  columns=['col1', 'col2', 'col3'])

# Create from dictionary
df = pd.DataFrame({'col1': ['A', 'B', 'C'],
                   'col2': [1, 2, 3]})

# Create from Series
col1 = pd.Series(['A', 'B', 'C'])
col2 = pd.Series([1, 2, 3])

df = pd.concat([col1, col2], axis=1)
df.columns = ['col1', 'col2']

## Index

In [9]:
df = pd.DataFrame({'col1': [3, 1, 2],
                   'col2': ['A', 'B', 'C'],
                   'col3': [8, 5, 7],
                   'col4': [True, False, True]})
df

,col1,col2,col3,col4
0,3,A,8,True
1,1,B,5,False
2,2,C,7,True


In [10]:
# Set index - move columns 'left' to become an index
df = df.set_index(['col1', 'col3'])  # can have multiple indexes
df

,,col2,col4
col1,col3,,
3,8,A,True
1,5,B,False
2,7,C,True


In [11]:
# Reset index - move columns back to the 'right'
df = df.reset_index('col1')
df

,col1,col2,col4
col3,,,
8,3,A,True
5,1,B,False
7,2,C,True


## Subset

### Select Columns

In [ ]:
# Using column name
df['col 1!']          # if column name has spaces or special characters
df[['col1', 'col2']]  # multiple columns

# Using dot notation
df.col1

# Using loc function
df.loc[:, ['col1','col2']]

### Select Rows

#### By column value
Must use bitwise operators '&' (AND) and '|' (OR)

In [ ]:
# Select rows on condition
df[df['col'] == 5]
df[(df['col'] > 5) & (df['col'] < 10)]

# Select rows with nulls in specified column
df[df['col'].isnull()]

# Select rows without nulls in specified column
df[df['col'].notnull()]

#### By index label
Selecting rows by index is faster than '=='. Even faster if already sorted by index.

In [ ]:
df.loc[0]          # row with index label 0
df.loc[1:3]        # rows with index label 1 to 3 inclusive
df.loc['A']        # rows with index label 'A'
df.loc[['A','B']]  # rows with index label 'A' or 'B'
df.loc['A', 15]    # rows where first index is 'A' and second index is 15

#### By position

In [ ]:
df.iloc[0]         # first row
df.iloc[1:5]       # rows at position 1 to 4

### Select Columns and Rows

In [ ]:
# col1 and col2, rows where col3<4
df[['col1','col2']][df['col3']<4]

# col1 and col2, rows with index label 'A' or 'B'
df[['col1','col2']].loc[['A', 'B']]

# col1 and col2, rows at position 1 to 3
df[['col1','col2']].iloc[1:4]

## Sort

In [ ]:
# Sort by value ascending
df.sort_values(by='col1')

# Sort by value descending
df.sort_values(by='col1', ascending=False)

In [15]:
# Sort by index
df = pd.DataFrame({'col1': [3, 1, 2],
                   'col2': [1, 2, 3]}).set_index('col1')
df.sort_index()

,col2
col1,
1,2
2,3
3,1


## Append

In [ ]:
# Keep duplicate rows
pd.concat([df1, df2])

# Drop duplicate rows
pd.concat([df1, df2]).drop_duplicates()

# Drop duplicate rows based on specified columns, keeping first occurrence
pd.concat([df1, df2]).drop_duplicates(subset=['col1', 'col2'])

# Drop duplicate rows based on specified columns, keeping last occurrence
pd.concat([df1, df2]).drop_duplicates(subset=['col1', 'col2'], keep='last')

## Join

In [ ]:
# Column to column inner join
pd.merge(df1, df2,
         left_on='coll', right_on='colr',
         how='inner',
         suffixes=['', '_r'])  # column name suffixes for duplicates

# Index to index left join
pd.merge(df1, df2,
         left_index=True, right_index=True,
         how='left',
         suffixes=['', '_r'])

# Column to index outer join
pd.merge(df1, df2,
         left_on='coll', right_index=True,
         how='outer',
         suffixes=['', '_r'])

## Reshape

### Pivoting

In [125]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two', 'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'z', 'x', 'x', 'z', 'z']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,z
2,one,C,3,x
3,two,A,4,x
4,two,B,5,z
5,two,C,6,z


In [33]:
# Pivot - Transpose tall to wide
df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


In [46]:
# Pivot - Transpose tall to wide with aggregation of values
df.pivot_table(index='foo', columns='zoo', values='baz', aggfunc=np.sum)

zoo,x,z
foo,,
one,4,2
two,4,11


### Stacking/Unstacking

Dataframes require both left and top index. If you stack/unstack too much, it will become a Series.

In [72]:
df = pd.DataFrame({'foo': ['one', 'one', 'two', 'two'],
                   'bar': ['A', 'B', 'A', 'B'],
                   'baz': [1, 3, 5, 7],
                   'zoo': [2, 4, 6, 8]}).set_index(['foo', 'bar'])
df

baz  zoo
foo bar          
one A      1    2
    B      3    4
two A      5    6
    B      7    8

In [74]:
# Move innermost left index 'up' to innermost top index - Wider
df_unstacked = df.unstack()
df_unstacked

baz    zoo   
bar   A  B   A  B
foo              
one   1  3   2  4
two   5  7   6  8

In [75]:
# Move innermost top index 'down' to innermost left index - Taller
df_unstacked.stack()

baz  zoo
foo bar          
one A      1    2
    B      3    4
two A      5    6
    B      7    8